In [ ]:
#read how many imagaes are there in a folder called raw/images/all_objects

#create pandas dataframe with below columns
#image_name, image_path, col_number, row_number, label
#Take col_number and row_number starting from 1,1 at top left corner and 
# increasing to right and down respectively till 8,8
import os
import pandas as pd
from PIL import Image, ImageDraw

def draw_grid_on_image(image_path, tile_width, tile_height, output_path):
    # Open the image
    img = Image.open(image_path)
    draw = ImageDraw.Draw(img)
    img_width, img_height = img.size

    # Draw vertical lines
    for x in range(0, img_width, tile_width):
        draw.line([(x, 0), (x, img_height)], fill='red', width=2)

    # Draw horizontal lines
    for y in range(0, img_height, tile_height):
        draw.line([(0, y), (img_width, y)], fill='red', width=2)
    
    # Add numbers to each tile starting from (1,1) at top-left
    num_tiles_x = img_width // tile_width
    num_tiles_y = img_height // tile_height
    for i in range(num_tiles_x):
        for j in range(num_tiles_y):
            x = i * tile_width + tile_width // 2
            y = j * tile_height + tile_height // 2
            cell_no = (j)*8 + (i+1)
            # draw.text((x+1, y+1), f'{j+1},{i+1}', fill='blue')
            draw.text((x+1, y+1), f'{cell_no}', fill='blue')

    # Save the image with grid
    img.save(output_path)

image_folder = '../data/raw/images/all_objects'
output_grid_image_path = '../data/processed/tiled_images'
data = []
for image_name in os.listdir(image_folder):
    if image_name.endswith(('.png', '.jpg', '.jpeg')):
        image_path = os.path.join(image_folder, image_name)
        output_path = os.path.join(output_grid_image_path, image_name)        
        draw_grid_on_image(image_path, 100, 75, output_path)
        for col in range(8):
            for row in range(8):                
                data.append({
                    'image_name': image_name,                    
                    'col_number': col + 1,
                    'row_number': row + 1,
                    'label': 1
                })
df = pd.DataFrame(data)
print(df.head())
print(f'Total records: {len(df)}')
#save dataframe to csv
df.to_csv('../data/processed/image_tiles_labels.csv', index=False)

      image_name  col_number  row_number  label
0  cricket_1.jpg           1           1      1
1  cricket_1.jpg           1           2      1
2  cricket_1.jpg           1           3      1
3  cricket_1.jpg           1           4      1
4  cricket_1.jpg           1           5      1
Total records: 768


PermissionError: [Errno 13] Permission denied: '../data/processed/image_tiles_labels.csv'

In [ ]:
# Cricket Object Detection in Tiled Images
# This cell scans through tiled images and detects cricket objects (bat, ball, stump)
# Each image is divided into a grid of 64 tiles (8x8), numbered from 1 to 64

import os
import pandas as pd
from PIL import Image
import numpy as np

def extract_tile(image, tile_number, tile_width=100, tile_height=75):
    """
    Extract a specific tile from the gridded image.
    Tile numbering: 1-8 (top row), 9-16 (second row), ..., 57-64 (bottom row)
    Formula: tile_number = row * 8 + col + 1 (where row and col start from 0)
    """
    # Convert tile number to row and column (0-indexed)
    tile_idx = tile_number - 1  # Convert to 0-indexed
    row = tile_idx // 8
    col = tile_idx % 8
    
    # Calculate pixel coordinates
    left = col * tile_width
    top = row * tile_height
    right = left + tile_width
    bottom = top + tile_height
    
    # Extract and return the tile
    tile = image.crop((left, top, right, bottom))
    return tile

def analyze_tile_simple(tile_image):
    """
    Simple analysis of a tile to detect potential cricket objects.
    This is a basic implementation using color and shape heuristics.
    
    Returns:
        list: Detected objects in the tile (e.g., ['bat'], ['ball'], ['stump'], or [])
    """
    # Convert to numpy array for analysis
    img_array = np.array(tile_image)
    
    # Basic heuristics (this is a simplified approach)
    # In a real implementation, you would use:
    # - Pre-trained object detection models (YOLO, Faster R-CNN)
    # - Custom trained models with labeled cricket object data
    # - More sophisticated computer vision techniques
    
    detected_objects = []
    
    # Calculate basic statistics
    mean_color = img_array.mean(axis=(0, 1))
    std_color = img_array.std(axis=(0, 1))
    
    # These thresholds are placeholders and would need to be tuned
    # based on actual cricket object characteristics
    # For now, we'll use a simple approach that flags potential objects
    
    # Note: This is a demonstration. For actual detection, manual labeling
    # or a trained model would be needed
    
    return detected_objects  # Empty for now, needs manual labeling or trained model

def scan_tiled_images(tiled_images_folder, output_csv_path=None):
    """
    Scan through all tiled images and detect cricket objects in each tile.
    
    Args:
        tiled_images_folder: Path to folder containing tiled images
        output_csv_path: Path to save the detection results
    
    Returns:
        DataFrame with columns: image_name, tile_number, detected_objects
    """
    results = []
    
    # Get all image files
    image_files = [f for f in os.listdir(tiled_images_folder) 
                   if f.endswith(('.png', '.jpg', '.jpeg'))]
    
    print(f"Scanning {len(image_files)} tiled images...")
    
    for image_name in image_files:
        image_path = os.path.join(tiled_images_folder, image_name)
        img = Image.open(image_path)
        
        print(f"\nProcessing: {image_name}")
        
        # Scan through all 64 tiles
        for tile_num in range(1, 65):
            tile = extract_tile(img, tile_num)
            detected = analyze_tile_simple(tile)
            
            # Record results even if no objects detected
            # This helps in manual review later
            results.append({
                'image_name': image_name,
                'tile_number': tile_num,
                'detected_objects': ','.join(detected) if detected else 'none'
            })
    
    # Create DataFrame
    df = pd.DataFrame(results)
    
    # Save to CSV if path provided
    if output_csv_path:
        df.to_csv(output_csv_path, index=False)
        print(f"\nResults saved to: {output_csv_path}")
    
    return df

def manual_label_tiles(tiled_images_folder, output_csv_path):
    """
    Interactive function to manually label tiles with cricket objects.
    This is useful when automatic detection is not available.
    """
    print("Manual Labeling Mode")
    print("="*50)
    print("Instructions:")
    print("1. For each image, you'll need to identify tiles containing objects")
    print("2. Objects to detect: bat, ball, stump")
    print("3. Tiles are numbered 1-64 (top-left to bottom-right, row by row)")
    print("="*50)
    
    # This would require interactive input which doesn't work well in notebooks
    # Instead, we'll create a template CSV for manual editing
    
    image_files = [f for f in os.listdir(tiled_images_folder) 
                   if f.endswith(('.png', '.jpg', '.jpeg'))]
    
    template_data = []
    for image_name in image_files:
        for tile_num in range(1, 65):
            template_data.append({
                'image_name': image_name,
                'tile_number': tile_num,
                'contains_bat': 0,  # 1 if bat present, 0 otherwise
                'contains_ball': 0,  # 1 if ball present, 0 otherwise
                'contains_stump': 0,  # 1 if stump present, 0 otherwise
                'notes': ''  # Any additional observations
            })
    
    df = pd.DataFrame(template_data)
    df.to_csv(output_csv_path, index=False)
    print(f"\nTemplate created at: {output_csv_path}")
    print("Please edit this CSV file to mark which tiles contain which objects.")
    print("Set contains_bat/ball/stump to 1 where objects are present.")
    
    return df

# Main execution
tiled_images_folder = '../data/processed/tiled_images'
output_detection_csv = '../data/processed/cricket_object_detections.csv'
output_template_csv = '../data/processed/cricket_object_labeling_template.csv'

print("CRICKET OBJECT DETECTION SYSTEM")
print("="*70)
print("\nApproach: Given the complexity of object detection, two options are provided:")
print("\n1. AUTOMATIC DETECTION (Limited):")
print("   - Uses basic image analysis")
print("   - May not be accurate without a trained model")
print("\n2. MANUAL LABELING TEMPLATE (Recommended):")
print("   - Creates a CSV template for manual object labeling")
print("   - More accurate but requires human review")
print("\n" + "="*70)

# Option 1: Attempt automatic detection (will be limited without trained model)
print("\n[Running automatic scan...]")
detection_df = scan_tiled_images(tiled_images_folder, output_detection_csv)
print(f"\nScanned {len(detection_df)} total tiles across all images.")
print("\nSample results:")
print(detection_df.head(20))

# Option 2: Create manual labeling template
print("\n" + "="*70)
print("\n[Creating manual labeling template...]")
template_df = manual_label_tiles(tiled_images_folder, output_template_csv)

print("\n" + "="*70)
print("\nNEXT STEPS:")
print("1. Review the template CSV file")
print("2. For each image, open it and identify which tiles contain objects")
print("3. Mark the corresponding rows in the CSV (set contains_* columns to 1)")
print("4. The marked CSV will serve as ground truth for object locations")
print("\nNote: For production use, consider training an object detection model")
print("with labeled cricket images (YOLO, Faster R-CNN, etc.)")
print("="*70)